# Problem 1: API call limits.
The wikipedia api limits the results to 500 responses per call. Some of the sample categories have more than 500 pages.  The downloading function was made recurrsive so it could download all the categories.

The function to download categories from the project 6 solution is below. see  the file 'notebooks/lib/wiki_module.py'

In [ ]:
def query_category(category):
    """
    
    Takes category name and returns a dictionary with category id and a list of pages (page id and title) associated with that category.
    
    """
    base_url = "https://en.wikipedia.org/w/api.php"
    
    param_one = cat_param_id(category)
    query_one = param_query(param_one)
    response_one = requests.get(base_url+query_one)
    categoryid = unidecode(response_one.json()['query']['pages'].keys()[0])
    
    param_two = cat_param_pages(category)
    query_two = param_query(param_two)
    response_two = requests.get(base_url+query_two)
    pages = response_two.json()['query']['categorymembers']
    
    return {'categoryid':categoryid,'pages':[{'pageid':page['pageid'],'title':(unidecode(
                        page['title']))} for page in pages]}

### Updated code with recurrsion.
The module for calling wikipedia was significantly altered. see the file 'notebooks/lib/download_mine.py'. Lines 8-9 check if recurrsion is needed. Lines 17-19 make the recurrsive call.

In [ ]:
def wikipedia_get_pages_for_category(category, **kwargs):
    response_list = []
    params = { 'action' : 'query',
               'format' : 'json',
               'list' : 'categorymembers',
               'cmlimit' : 'max'
             }
    if kwargs.get('continue_string',False):
        params['cmcontinue'] = kwargs['continue_string']
        
    params['cmtitle'] = 'Category:'+wikipedia_page_format(category)
       
    query_param_string = create_query_param_string(params)
    response = get(base_url+query_param_string)
    response_list = response.json()['query']['categorymembers']    
    
    if 'continue' in response.json().keys():
        response_list.extend(wikipedia_get_pages_for_category
                             (category, continue_string = response.json()['continue']['cmcontinue'] ))
    return response_list